# Data Preparaton 
Zusammenführung alle Excel-Sheets

In [2]:
import pandas as pd

In [3]:
file_path = "original_data/Results.xlsx"

In [4]:
from IPython.display import display, HTML

# CSS-Styles für alle DataFrames festlegen
display(HTML("""
<style>
    table.dataframe td {
        min-width: 90px;

    }
</style>
"""))

In [5]:
def replace_spelling_mistakes(df):
    """
    Ersetzt bestimmte Werte und Spaltennamen in einem DataFrame.
    
    Korrekturen:
    1. Ersetzt `",,"` durch `","`.
    2. Ersetzt `Thermogram Characterization` durch `Peak Signal Characterisation` in den Spaltennamen.

    Parameter:
    df (pd.DataFrame): Das DataFrame, das bearbeitet werden soll.

    Rückgabe:
    pd.DataFrame: Das korrigierte DataFrame.
    """
    # Ersetze `",,"` durch `","` im gesamten DataFrame
    df = df.replace(to_replace=",,", value=",", regex=True)
    
    # Ersetze `Thermogram Characterization` im gesamten DataFrame
    df = df.replace(to_replace="Thermogram Characterization", value="Peak Signal Characterisation", regex=True)
    
    # Ersetze `"m,1"` durch `"m"` im gesamten DataFrame
    df = df.replace(to_replace="m,1", value="m", regex=True)
    
    # Ersetze `"c,1"` durch `"c"` im gesamten DataFrame
    df = df.replace(to_replace="c,1", value="c", regex=True)
    
    return df

In [6]:
def split_raw_sheet(df):
    tables = {}  # Dictionary zur Speicherung der Tabellen
    table_start_col = None  # Startspalte der aktuellen Tabelle
    table_name = None  # Name der aktuellen Tabelle

    # Durchlaufen der Spalten
    for col in df.columns:
        if pd.notna(df.iloc[0, col]):  # Tabellenname gefunden in Zeile 1
            if table_start_col is not None:
                # Speichere die vorherige Tabelle
                data_start = 3  # Daten starten in Zeile 4
                table_data = df.iloc[data_start:, table_start_col:col].reset_index(drop=True)
                
                # Header aus Zeile 3 setzen
                header_row = df.iloc[data_start - 1, table_start_col:col].values
                table_data.columns = header_row  # Setze Header ohne dropna
                
                tables[table_name] = table_data  # Speichern der Tabelle
            
            # Start einer neuen Tabelle
            table_start_col = col
            table_name = df.iloc[0, col]
    
    # Letzte Tabelle speichern
    if table_start_col is not None and table_name:
        data_start = 3  # Daten starten in Zeile 4
        table_data = df.iloc[data_start:, table_start_col:].reset_index(drop=True)
        
        # Header aus Zeile 3 setzen
        header_row = df.iloc[data_start - 1, table_start_col:].values
        table_data.columns = header_row  # Setze Header ohne dropna
        
        tables[table_name] = table_data  # Speichern der Tabelle
    
    return tables

In [7]:
def _fill_column_names_with_previous(dframe):
    """
    Füllt NaN-Spaltennamen mit dem vorherigen gültigen Spaltennamen und hängt ggf. die Werte aus der ersten Zeile an.

    Args:
        dframe (pd.DataFrame): Das Eingabe-DataFrame.

    Returns:
        pd.DataFrame: Das angepasste DataFrame mit neuen Spaltennamen.
    """
    # Kopiere das DataFrame, um es nicht direkt zu verändern
    df = dframe.copy()
    
    # Erstelle neue Spaltennamen durch Auffüllen von NaN mit vorherigen Werten
    filled_columns = []
    previous_col = None
    
    for i, col in enumerate(df.columns):
        if pd.notna(col):
            previous_col = col  # Speichere gültigen Spaltennamen
        filled_columns.append(previous_col)
    
    # Füge die Werte aus der ersten Zeile als Suffix hinzu
    new_columns = []
    for i, col_name in enumerate(filled_columns):
        suffix = df.iloc[0, i]  # Wert aus der ersten Zeile
        if pd.notna(suffix):
            new_columns.append(f"{col_name}_{suffix}")  # Anhängen des Suffix
        else:
            new_columns.append(col_name)  # Behalte nur den Hauptnamen, wenn Suffix NaN
    
    # Setze die neuen Spaltennamen und entferne die erste Zeile
    df.columns = new_columns
    df = df.drop(index=0).reset_index(drop=True)
    
    return df

In [8]:
def _remove_unwanted_rows(df, column_name, unwanted_values):
    """
    Entfernt Zeilen, die bestimmte Werte in einer Spalte enthalten.
    """
    return df[~df[column_name].isin(unwanted_values)]

In [9]:
def _combine_sample_names(df, column_name, main_substring = "PE"):
    """
    Kombiniert Hauptnamen mit den Unterzeilen und löscht Zeilen ohne Unterstrich im angegebenen Feld.

    Parameter:
    df (pd.DataFrame): Das DataFrame, das bearbeitet werden soll.
    column_name (str): Der Name der Spalte, die angepasst und gefiltert wird.

    Rückgabe:
    pd.DataFrame: Ein gefiltertes DataFrame ohne Hauptnamenzeilen.
    """
    current_main_name = None
    new_sample_names = []

    # 1. Hauptnamen mit Unterwerten kombinieren
    for value in df[column_name]:
        if isinstance(value, str) and main_substring in value:  # Hauptname erkannt
            current_main_name = value
            new_sample_names.append(value)  # Hauptzeile bleibt gleich
        elif current_main_name:  # Unterzeilen kombinieren mit Hauptname
            new_name = f"{current_main_name}_{value}"
            new_sample_names.append(new_name)
        else:  # Für NaN oder sonstige Fälle
            new_sample_names.append(value)

    # Kopie erstellen und aktualisieren, um Warning zu vermeiden
    df_copy = df.copy()
    df_copy.loc[:, column_name] = new_sample_names

    # 2. Zeilen löschen, die keinen Unterstrich enthalten
    df_filtered = df_copy[df_copy[column_name].str.contains("_", na=False)]

    return df_filtered

In [10]:
def concat_tables(pe_tables_dict, sheet_name = "PE"):
    """
    Transformiert alle Tabellen im Dictionary und kombiniert sie in ein einziges DataFrame.
    Fügt eine neue Spalte 'Segment' hinzu, die den Key (table_name) enthält.

    Schritte:
    1. Füllt Spaltennamen.
    2. Löscht unerwünschte Zeilen basierend auf 'Sample Name'.
    3. Kombiniert 'Sample Name' mit Unterwerten.
    4. Fügt eine neue Spalte 'Segment' mit dem Dictionary-Key hinzu.
    
    Parameter:
    pe_tables_dict (dict): Dictionary mit Tabellen (DataFrames).

    Rückgabe:
    pd.DataFrame: Ein kombiniertes DataFrame mit einer zusätzlichen 'Segment'-Spalte.
    """
    combined_df = []

    for table_name, table_df in pe_tables_dict.items():
        # Schritt 1: Spaltennamen füllen
        table_df = _fill_column_names_with_previous(table_df)
        
        if 'Sample Name' in table_df.columns:
            # Schritt 2: Unerwünschte Zeilen entfernen
            table_df = _remove_unwanted_rows(
                table_df, 'Sample Name', ['x̅', 'σ', 'σrel in %']
            )
            
            # Schritt 3: Sample Names kombinieren
            table_df = _combine_sample_names(table_df, 'Sample Name', sheet_name.strip()) # strip WICHTIG!

        # Schritt 4: 'Segment'-Spalte hinzufügen
        table_df['Segment'] = table_name

        # DataFrame zur kombinierten Liste hinzufügen
        combined_df.append(table_df)

    # Alle DataFrames vertikal kombinieren
    return pd.concat(combined_df, ignore_index=True)

In [11]:
def dataframe_sheet_loader(file_path, sheet_name, key_substring="Heating"): # alternativ: "Cooling"
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    df_raw = replace_spelling_mistakes(df_raw)
    df_dict = split_raw_sheet(df_raw)
    
    # Dictionary nach Keys eingrenzen 
    df_dict = {key: value for key, value in df_dict.items() if key_substring in key}
    return concat_tables(df_dict, sheet_name)

## Test der Funktionen

### 0-PE

In [14]:
sheet_name = "0-PE"
dataframe_sheet_loader(file_path, sheet_name, "Heating").head(10)

,Sample Name,Mass in mg,TStart in °C,TEnd in °C,Tm in °C,∆Hm in Jg-1,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Segment
0,0-PE-1_1,1.17,55.1,118,110.3,109.6,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
1,0-PE-1_2,1.22,55,119.8,108.5,106,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
2,0-PE-1_3,1.19,55,118,110.1,110.2,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
3,0-PE-1_4,1.27,55,118.22,110.7,108,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
4,0-PE-1_5,1.41,55,118.42,110.6,109.4,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
5,0-PE-2_1,1.48,55,121.81,108.6,113.9,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
6,0-PE-2_2,1.44,55,120.08,109.3,116.3,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
7,0-PE-2_3,1.42,55,120.48,109.9,114.1,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
8,0-PE-2_4,1.49,55,120.06,110.7,117.6,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating
9,0-PE-2_5,1.42,55,122.03,109.6,118.4,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating


### 3-EBT-PET

In [16]:
sheet_name = "3-EBT-PET"
dataframe_sheet_loader(file_path, sheet_name).head(10)

,Sample Name,Mass in mg,TStart in °C,TEnd in °C,Tm in °C,∆Hm in Jg-1,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Segment,Peak Signal Characterisation_Quasibimodal
0,3-EBT-PET-1_1,1.41,213.73,255.6,240.1,38.83,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
1,3-EBT-PET-1_2,1.16,213.6,252.08,240.2,38.33,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
2,3-EBT-PET-1_3,1.47,213.86,253.99,239.8,38.99,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
3,3-EBT-PET-1_4,1.3,213.185,250.7,240,37.82,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
4,3-EBT-PET-1_5,1.21,212.5,254.59,239.5,37.35,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
5,3-EBT-PET-2_1,1.38,212.7,254.7,240.1,38.06,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
6,3-EBT-PET-2_2,1.26,212.7,252.75,240.2,37.56,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
7,3-EBT-PET-2_3,1.41,212.68,254.38,240.9,37.95,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
8,3-EBT-PET-2_4,1.45,212.76,254.58,240.6,38.47,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN
9,3-EBT-PET-2_5,1.34,212.7,254.7,240.1,38.31,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Heating,NaN


## Gesamte Excel (ohne Legende) 
### für Heating

In [18]:
sheet_names = pd.ExcelFile(file_path).sheet_names
sheet_names

['Legend',
 '0-PE',
 '1-UV-PE',
 '2-UV-PE ',
 '3-UV-PE ',
 '1-EBT-PE ',
 '2-EBT-PE  ',
 '3-EBT-PE',
 '0-PET',
 '1-UV-PET',
 '2-UV-PET',
 '3-UV-PET ',
 '1-EBT-PET  ',
 '2-EBT-PET',
 '3-EBT-PET']

In [19]:
def dataframe_excel_loader(file_path, start_sheet_name, key_substring= "Heating"):
    """
    Lädt und verarbeitet alle Sheets aus einer Excel-Datei ab einem bestimmten Startsheet.
    
    Parameter:
    file_path (str): Pfad zur Excel-Datei.
    start_sheet_name (str): Name des ersten Sheets, ab dem die Verarbeitung beginnt.
    
    Rückgabe:
    pd.DataFrame: Ein kombiniertes DataFrame mit allen verarbeiteten Sheets.
    """
    # Alle Sheet-Namen aus der Excel-Datei abrufen
    sheet_names = pd.ExcelFile(file_path).sheet_names

    # Index des Start-Sheets finden
    try:
        start_index = sheet_names.index(start_sheet_name)
    except ValueError:
        raise ValueError(f"Sheet '{start_sheet_name}' nicht in der Excel-Datei gefunden.")
    
    # Nur die relevanten Sheets ab dem Startindex auswählen
    sheets_to_process = sheet_names[start_index:]
    
    # Verarbeite jedes Sheet mit dataframe_sheet_loader
    combined_sheets = [dataframe_sheet_loader(file_path, sheet, key_substring) for sheet in sheets_to_process]
    
    # Kombiniere alle DataFrames in ein einziges DataFrame
    return pd.concat(combined_sheets, ignore_index=True)


In [20]:
# Beispiel: Datei und Startsheet angeben
start_sheet_name = "0-PE"

# Verarbeite alle relevanten Sheets
df_data = dataframe_excel_loader(file_path, start_sheet_name)
# Ergebnis anzeigen
df_data

,Sample Name,Mass in mg,TStart in °C,TEnd in °C,Tm in °C,∆Hm in Jg-1,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Segment,"Tm,2 in °C",Peak Signal Characterisation_Quasibimodal
0,0-PE-1_1,1.17,55.1,118,110.3,109.6,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating,NaN,NaN
1,0-PE-1_2,1.22,55,119.8,108.5,106,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating,NaN,NaN
2,0-PE-1_3,1.19,55,118,110.1,110.2,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating,NaN,NaN
3,0-PE-1_4,1.27,55,118.22,110.7,108,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating,NaN,NaN
4,0-PE-1_5,1.41,55,118.42,110.6,109.4,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Heating,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,3-EBT-PET-4_1,1.18,205.46,254.29,239.9,36.62,NaN,NaN,x,s,NaN,NaN,NaN,NaN,2nd Heating,NaN,NaN
587,3-EBT-PET-4_2,1.27,201.99,251.99,239.9,36.95,NaN,NaN,x,s,NaN,NaN,NaN,NaN,2nd Heating,NaN,NaN
588,3-EBT-PET-4_3,1.3,202.37,251.33,240,36.87,NaN,NaN,x,s,NaN,NaN,NaN,NaN,2nd Heating,NaN,NaN
589,3-EBT-PET-4_4,1.21,198.86,256.1,239.3,36.3,NaN,NaN,x,s,NaN,NaN,NaN,NaN,2nd Heating,NaN,NaN


In [21]:
df_data.to_csv("data/00_Data_Heating.csv", index=False)

### für Cooling

In [23]:
# Beispiel: Datei und Startsheet angeben
start_sheet_name = "0-PE"

# Verarbeite alle relevanten Sheets
df_data = dataframe_excel_loader(file_path, start_sheet_name, "Cooling")
# Ergebnis anzeigen
df_data

,Sample Name,Mass in mg,TStart in °C,TEnd in °C,Tc in °C,∆Hc in Jg-1,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Segment,"Tc,2 in °C"
0,0-PE-1_1,1.17,36,99.77,93.3,-119.1,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Cooling,NaN
1,0-PE-1_2,1.22,36,99.85,93.4,-120.8,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Cooling,NaN
2,0-PE-1_3,1.19,36,99.29,93.4,-119.5,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Cooling,NaN
3,0-PE-1_4,1.27,36,99.3,93.2,-121.3,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Cooling,NaN
4,0-PE-1_5,1.41,36,98.71,93.4,-119,NaN,x,NaN,l,NaN,x,NaN,NaN,1st Cooling,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,3-EBT-PET-4_1,1.18,163.2,208.05,188.5,-41.04,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Cooling,NaN
292,3-EBT-PET-4_2,1.27,163.2,208.5,189.6,-43.18,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Cooling,NaN
293,3-EBT-PET-4_3,1.3,162.8,207.63,190.2,-43.28,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Cooling,NaN
294,3-EBT-PET-4_4,1.21,163.2,208.39,190.4,-42.31,NaN,NaN,x,s,NaN,NaN,NaN,NaN,1st Cooling,NaN


In [24]:
df_data.to_csv("data/00_Data_Cooling.csv", index=False)